In [19]:
"""
Test stacking with labels and some convolutions.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os
import utils
import tensorflow as tf
from config import cfg
import conv_bodies
import modeling
import matplotlib.pyplot as plt
    

model, weights_file, output_dir = modeling.create_model()
model.set_dataset('train_data/', 'train_labels/')


with model._graph.as_default():
    init = (tf.global_variables_initializer(), 
            tf.local_variables_initializer(), 
            model.dataset[1].make_initializer(model.dataset[0]))

    model.tensor, model.labels = utils.stack_batch_into_tensor(model.dataset[0], model._graph, train=True) # Normally use model.train

with tf.Session(graph=model._graph) as sess:
    sess.run(init)
    print('Tensor shape:', model.tensor.shape)
    with tf.variable_scope('conv1', reuse=tf.AUTO_REUSE):
        model.Conv(3, 3, 64, padding='SAME', strides=[1,1,1,1])
        model.Relu()
    print('Tensor shape:', model.tensor.shape)
    with tf.variable_scope('conv2', reuse=tf.AUTO_REUSE):
        model.Conv(3, 64, 64, padding='SAME', strides=[1,1,1,1])
        model.Relu()
        # Strides[0] must be 1, else
        # UnimplementError Pooling is not yet supported on the batch dimension
        model.MaxPool(ksize=[1,1,2,1], strides=[1,2,2,2], padding='SAME')
    print('Tensor shape:', model.tensor.shape)

    
    

Tensor shape: (10, 224, 224, 3)
Tensor shape: (10, 224, 224, 64)
Tensor shape: (10, 112, 112, 32)


In [13]:
"""
Test loading datasets and iterating on them.
"""

graph = tf.Graph()
with graph.as_default():   
    dataset, iterator = utils.load_dataset('train_data/', graph, 'train_labels/')
    tensor = iterator.get_next()
    next_element = iterator.get_next()
    init = iterator.make_initializer(dataset)

with tf.Session(graph=graph) as sess:
    # Initialize the iterator on the training data
    sess.run(init)
    tensor_list = [sess.run(tensor)[0]]
    # get each element of the training dataset until the end is reached
    while True:
        try:
            elem = sess.run(next_element)
            tensor_list += [elem[0]]
        except tf.errors.OutOfRangeError:
            break
    tensor = tf.stack(tensor_list, axis=0)
    print('Tensor shape:', tensor.shape)
    print('Batch size:', tensor.shape[0])

Tensor shape: (10, 224, 224, 3)
Batch size: 10


In [7]:
"""
Test stacking without labels and a convolutions.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os
import utils
import tensorflow as tf
from config import cfg
import conv_bodies
import modeling
    

model, weights_file, start_iter, checkpoints, output_dir = modeling.create_model()
model.set_dataset('train_data/')


model.tensor = utils.stack_batch_into_tensor(model.dataset[0], model._graph, train=False) # Normally use model.train

print('Tensor shape:', model.tensor.shape)
with model._graph.as_default():
    init = (tf.global_variables_initializer(), 
            tf.local_variables_initializer(), 
            model.dataset[1].make_initializer(model.dataset[0]))

with tf.Session(graph=model._graph) as sess:
    sess.run(init)
    with tf.variable_scope('conv', reuse=tf.AUTO_REUSE):
        model.Conv(3, 3, 64, padding='SAME', strides=[1,1,1,1])
    print('Tensor shape:', model.tensor.shape)
    
    

Tensor shape: (10, 224, 224, 3)
Tensor shape: (10, 224, 224, 64)
